In [27]:
import numpy as np
import matplotlib.pyplot as plt

import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# pio.templates.default = "ggplot2"
pio.templates.default = "seaborn"
# pio.templates.default = "plotly"

# Simplifcation des entrées

Dans ce notebook, nous voulons simplifier les données et passer d'une entrée à 48 cannaux à une entrée à 12 cannaux. Nous ne somme intéressés que par le bord contenant la source et celui en face de lui.

## Test de reshape

In [28]:
X = np.zeros((2, 3,2,2))
X[0, 1,1,:] = 1
X

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [1., 1.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]])

In [29]:
X_ = X.reshape(2,-1)
X_

array([[0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

L'ordre est clair: 4 pour la source, 4 pour les bords, 3 pour E,F et T
Pour passer à 48, on à:
- d'abord la source 0, puis le bord 0, et puis ses 3 cannaux,
- ensuite on à le bord 1, puis ses 3 cannaux
Autrement dit, on a une nouvelle simulation tous les 12 cannaux. Pour chaque simulation, je veux identifier la source (et donc le signal en face). Je peux juste analyser les énergies, qui apparaissent tous les 3 cannaux.

In [30]:

X = np.load('../data/test/X.npy')[:1]
X.shape

(1, 64, 64, 48)

In [31]:
# for src in [0,12,24,36]:
for src in [12]:

    print('Simulation #', 1 + src%12)

    fig = make_subplots(rows=3, cols=3)

    ## Dictionnary mapping data to its plot position
    plot_coord = {}
    plot_coord[0] = (1,2)
    plot_coord[3] = (3,2)
    plot_coord[6] = (2,1)
    plot_coord[9] = (2,3)

    for en in [0, 3, 6, 9]:
        if en==0:
            print('Énergie pour sur le bord du haut')
        elif en==3:
            print('Énergie pour sur le bord du bas')
        elif en==6:
            print('Énergie pour sur le bord gauche')
        else:
            print('Énergie pour sur le bord de droite')

        fig.add_trace(go.Heatmap(z=X[0,:,:,src+en], colorscale='plasma'), row=plot_coord[en][0], col=plot_coord[en][1])

    fig.update_layout(width=900, height=900, margin=dict(l=20, r=20, t=20, b=20))
    fig.update_traces(showscale=False)
    fig.show()

Simulation # 1
Énergie pour sur le bord du haut
Énergie pour sur le bord du bas
Énergie pour sur le bord gauche
Énergie pour sur le bord de droite


In [32]:
## Detect the source sine wave: subtract by itself to find almost zero

source = X[0,:,:,0]
another_source = X[0,:,:,12+3]
# fig = px.imshow(source-another_source, title='A source minus a source')
fig = px.imshow(source-another_source, title='A source minus a source',zmin=source.min(), zmax=source.max())
print(source-another_source)
# fig.show()

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.40861765e-07  5.62010785e-07  1.84660686e-06 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-4.72450352e-04 -6.09701414e-04 -7.61484470e-04 ...  8.80751187e-05
   9.88737545e-05  1.07745496e-04]
 [-3.74740762e-04 -4.42342631e-04 -4.99386726e-04 ...  2.76589593e-05
   4.64461770e-05  6.19014736e-05]
 [-1.74504349e-04 -1.53830380e-04 -1.00278781e-04 ... -2.28015804e-05
  -4.01436275e-07  1.82653505e-05]]


La fonction ci-bas sipprime les deux signaux directement opposés à la source, ainsi que la source elle même. Le réseau de neurone va essayer de reconstituer la source lui-même !

In [33]:
## Extraction des sources

def compress_inputs(X):
    (nb_samples, width, height, channels) = X.shape
    assert (width, height, channels) == (64, 64, 48), 'Error on inputs shape'

    new_X = np.zeros((nb_samples, 64, 64, 12), dtype=np.float32)

    for src in [0, 12, 24, 36]:

        if src==0:          ## Source en haut, on prend le signal en bas
            new_X[:, :, :, 0:3] = X[:, :, :, 3:6]
        elif src==12:       ## Source en bas, on prend le signal en haut
            new_X[:, :, :, 3:6] = X[:, :, :, src:src+3]
        elif src==24:       ## Source à gauche, on prend le signal à droite
            new_X[:, :, :, 6:9] = X[:, :, :, src+9:src+12]
        else:               ## Source à droite, on prend le signal à gauche
            new_X[:, :, :, 9:12] = X[:, :, :, src+6:src+9]

    return new_X


In [34]:
## Test the data compression 

new_X = compress_inputs(X)

fig = make_subplots(rows=3, cols=3)

plot_coord = {}
plot_coord[0] = (1,2)
plot_coord[3] = (3,2)
plot_coord[6] = (2,1)
plot_coord[9] = (2,3)

for en in [0, 3, 6, 9]:
    fig.add_trace(go.Heatmap(z=new_X[0,:,:,en], colorscale='plasma'), row=plot_coord[en][0], col=plot_coord[en][1])

fig.update_layout(width=900, height=900, margin=dict(l=20, r=20, t=20, b=20))
fig.update_traces(showscale=False)
fig.show()

In [36]:
## Now, let's really transform the data

new_X_train = compress_inputs(np.load('../data/train/X.npy'))
np.save('../data/train/newX.npy', new_X_train)

new_X_val = compress_inputs(np.load('../data/val/X.npy'))
np.save('../data/val/newX.npy', new_X_val)

new_X_test = compress_inputs(np.load('../data/test/X.npy'))
np.save('../data/test/newX.npy', new_X_test)
